## This script ingests the Critical Infrastructure Service into the HydroVIS EGIS Database. ##

Importing necessary modules and helper functions

In [1]:
import requests
import geopandas as gdp
import sys
import os
from shapely.geometry import shape
from arcgis.gis import GIS

#Adding parent folder to the Python path to grab helper functions.
sys.path.append(os.path.dirname(os.path.abspath('')))

#Importing necessary helper functions.
from helper_functions.shared_functions import get_db_engine, move_data_from_viz_to_egis

The Critical Infrastructure point layer is hosted as a feature service for storage and accessibility purposes. To retreive it within this script, you'll have to login to your TI ArcGIS Online account. You must setup environment variables "EKS_EGIS_TI_USERNAME" and "EKS_EGIS_TI_PASSWORD" in the windows environment variables. Once done, you'll need to close/reopen your IDE for it to successfully find your environment variables. 

This data is from https://services2.arcgis.com/C8EMgrsFcRFL6LrL/arcgis/rest/services/All_Infrastructure_Merge/FeatureServer/0. This data can still be accessed by logging in to https://noaa.maps.arcgis.com/home/ and finding the "All_Infrastructure_Merge" feature service. This data is derived from FEMA infrastructure datasaets. 

To get an editable copy of this data, the services2.arcgis.com link was used to connect to a new server in ArcGIS Pro. Feature service -> feature service in the geoprocessing toolbox was used to save it as a new local geodatabase. Then, for future accessbility, the data was published as a feature service within the EKS EGIS TI portal with organization wide access. This feature service is then connected to below for ingestion to the egis database.

In [ ]:
#Defining constants
FEATURE_SERVICE_URL = "https://eks-maps-testing.water.noaa.gov/server/rest/services/Hosted/static_public_critical_infrastructure_points_fema/FeatureServer/0/"
DB_TABLE_NAME = "static_public_critical_infrastructure_points_fema"

#User Crednetials for the TI portal that contains the servce.
#You must setup environment variables EKS_EGIS_TI_USERNAME and EKS_EGIS_TI_PASSWORD
username = os.getenv("EKS_EGIS_TI_USERNAME")
password = os.getenv("EKS_EGIS_TI_PASSWORD")

if not username or not password:
    raise EnvironmentError("Environment variables EKS_EGIS_TI_USERNAME and/or EKS_EGIS_TI_PASSWORD are not set.")

#Logging into the testing ArcGIS Portal
print("Attempting to login...")
gis = GIS(url="https://eks-maps-testing.water.noaa.gov/server/", username=username, password=password, verify_cert=False)

if gis is None or gis.users.me is None:
    print("Login failed.")
else:
    print("Succesfully logged in as " + str(gis.properties.user.username))

#Retrieving the token
token = gis.session.auth.token

This section fetches the data from the ArcGIS Server. The arcgis API restricts the number of rows that can be fetched to 2000. To get around this, we implement pagination with a while loop to keep grabbing a maximum of 2000 rows until we have all of the data. It should take no more than a couple of minutes to grab the data and store the features in an array variable called all_features. You'll run this section once, inspect the printed statement to ensure you're getting all of the data, then run the next functions.

In [4]:
def fetch_feature_service_data(feature_service_url):
    #Pagination variables to deal with ArcGIS Feature Service row limit of 2000
    max_record_count = 2000 #Row request limit
    offset = 0
    all_features = []

    while True:
        #Fetches data from the ArcGIS Server and prints some of the resulting GeoJSON for testing
        query_url = f"{feature_service_url}/query?where=1%3D1&outFields=*&outSR=3857&f=geojson&token={token}&resultOffset={offset}&resultRecordCount={max_record_count}"

        #Request the data
        response = requests.get(query_url)
        response.raise_for_status() # Raise an error if the request fails

        #Parse it as a GeoJSON
        geojson = response.json()
        
        #Creating shapely geometries
        features = geojson.get('features', [])
        if not features:
            print("No more features found.")
            break
        
        #Append fetched feature to list
        all_features.extend(features)
        print(f"Fetched {len(features)} features. Total so far: {len(all_features)}")

        #Increment offset for next query
        offset += max_record_count
    
    print(f"Total features fetched: {len(all_features)}")   
    return all_features

This function takes the all_features array the previous function returns, and stores it in a GeoDataFrame that it returns as df. It also creates the necessary oid field that's used in the VPP. 

In [1]:
def create_dataframe_from_features(features):
    attributes = [f['properties'] for f in features]
    geom = [shape(f['geometry']) for f in features]

    #Create GeoDataFrame
    df = gdp.GeoDataFrame(attributes, geometry = geom, crs = 'EPSG:3857')

    #Add oid column
    df = df.copy()
    if "ObjectID" in df.columns:
        print("ObjectID column found. Copying to the new oid column.")
        df["oid"] = df["ObjectID"].astype(int)
    else:
        print("No ObjectID column found. Making it from scratch.")
        df["oid"] = range(1, len(df) +1)

    #Print to see final GeoDataFrame before exporting to PostGIS
    print("\nSample of the GeoDataFrame with the added 'oid' column:")
    print(df.head(5).to_string(index=False))

    #Print the column names
    print("\nColumns in the GeoDataFrame")
    print(df.columns.to_list())

    #Print the number of rows in the dataframe
    print("\nNumber of rows in datraframe: ")
    print(len(df))

    return df

Now we're ready to execute the fetch_feature_service_data function.

In [ ]:
#Fetch the data
print(f"Fetching data from: {FEATURE_SERVICE_URL}")
features = fetch_feature_service_data(FEATURE_SERVICE_URL)

In [ ]:
#Create GeoDataFrame from the downloaded features
df = create_dataframe_from_features(features)

In this section we actually save the data to our postgis database. You'll need to setup your environment variables with the host address, username and password for the viz and egis database. These can be found in the RDS and Secrets Manager tools within the AWS TI platform.

When writing to the viz database, the 'if_exists' clause is set to 'replace' by default. Use caution, as this will overwrite the table. If you have updated the reference feature service and want to add new rows to the table, set if_exists = 'append'. If you are running this and want to be sure the table exists, set if_exists = 'fail' which will stop the script from altering the current table. 

In [11]:
#Save to DB
df.to_postgis("static_public_critical_infrastructure_points_fema", get_db_engine('viz'), schema='dev', if_exists='replace')

After checking the viz database for any issues and reviewing the data in ArcGIS Pro, we can migrate the table from the viz database to the egis database.

In [ ]:
move_data_from_viz_to_egis('dev.static_public_critical_infrastructure_points_fema', 'reference.static_public_critical_infrastructure_points_fema')